<a href="https://colab.research.google.com/github/nehabindle/Data-Structures/blob/master/Yelp_Neha_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s gdrive/'Team Drives'/'Data Mining Team'/ gdata

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install contractions

In [0]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

Importing Libraries from Keras.


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

In [51]:
#Checking the version on Tensorflow
tf.__version__


'1.13.1'

In [52]:
#Checking the version of Keras
tf.keras.__version__


'2.2.4-tf'

Load Data


In [0]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('gdata/yelpHotelData.db')
x1 = connection.execute("select * FROM review")

In [0]:
description = []
Class = []

data = x1.fetchmany(10000)

for x in data:
  description.append(x[3])
  Class.append(x[8])
trainData = {'Class' : Class, 'Description' : description}
df_X = pd.DataFrame(trainData)

In [55]:
df_X['Class'].value_counts()

N     5078
NR    2461
YR    1681
Y      780
Name: Class, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_X['Description'],df_X['Class'], test_size=0.3, random_state=42)

In [58]:
#Checking the shape of Train and Test Data
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(7000,) (3000,)
(7000,) (3000,)


In [59]:
#Checking the length of train and test data
print("Train-set size: ", len(X_train))
print("Test-set size:  ", len(X_test))

Train-set size:  7000
Test-set size:   3000


In [0]:
#Combine into one data-set for some uses below.
data_text = X_train + X_test

In [61]:
#Printing an example to see how data looks like
X_train[1]

'The only place inside the Loop that you can stay for $55/night. Also, the only place you can have a picnic dinner and get a little frisky on the 17th floor roof and then wake up in your room the next morning to an army of ants going in on your picnic leftovers.'

In [62]:
y_train[1]

'N'

Tokenizer
A neural network cannot work directly on text-strings so we must convert it somehow. There are two steps in this conversion, the first step is called the "tokenizer" which converts words to integers and is done on the data-set before it is input to the neural network. The second step is an integrated part of the neural network itself and is called the "embedding"-layer, which is described further below.

We may instruct the tokenizer to only use e.g. the 10000 most popular words from the data-set.


In [0]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)


The tokenizer can then be "fitted" to the data-set. This scans through all the text and strips it from unwanted characters such as punctuation, and also converts it to lower-case characters. The tokenizer then builds a vocabulary of all unique words along with various data-structures for accessing the data.



In [64]:
#Tokenization of train data
%%time
tokenizer.fit_on_texts(X_train)

CPU times: user 1.09 s, sys: 8.28 ms, total: 1.1 s
Wall time: 1.11 s


In [65]:
#Tokenization of test data
%%time
tokenizer.fit_on_texts(X_test)

CPU times: user 476 ms, sys: 2.68 ms, total: 478 ms
Wall time: 482 ms


In [0]:
#set num_words=None above, and then it will automatically be set to the vocabulary-size here.


if num_words is None:
    num_words = len(tokenizer.word_index)

We can then inspect the vocabulary that has been gathered by the tokenizer. This is ordered by the number of occurrences of the words in the data-set. These integer-numbers are called word indices or "tokens" because they uniquely identify each word in the vocabulary.

In [67]:
tokenizer.word_index


{'the': 1,
 'and': 2,
 'a': 3,
 'to': 4,
 'i': 5,
 'was': 6,
 'of': 7,
 'in': 8,
 'for': 9,
 'is': 10,
 'it': 11,
 'that': 12,
 'but': 13,
 'my': 14,
 'with': 15,
 'on': 16,
 'you': 17,
 'this': 18,
 'we': 19,
 'hotel': 20,
 'room': 21,
 'at': 22,
 'have': 23,
 'they': 24,
 'not': 25,
 'were': 26,
 'had': 27,
 'are': 28,
 'so': 29,
 'there': 30,
 'be': 31,
 'as': 32,
 'very': 33,
 'here': 34,
 'great': 35,
 'if': 36,
 'out': 37,
 'from': 38,
 'all': 39,
 'place': 40,
 'me': 41,
 'our': 42,
 'like': 43,
 'good': 44,
 'one': 45,
 'or': 46,
 'up': 47,
 'get': 48,
 'just': 49,
 '\xa0the': 50,
 'nice': 51,
 'would': 52,
 'when': 53,
 'an': 54,
 '\xa0': 55,
 "it's": 56,
 'about': 57,
 'which': 58,
 'service': 59,
 'stay': 60,
 'really': 61,
 '\xa0i': 62,
 'time': 63,
 'no': 64,
 'some': 65,
 'night': 66,
 'rooms': 67,
 'their': 68,
 'can': 69,
 'location': 70,
 'food': 71,
 'only': 72,
 'more': 73,
 'staff': 74,
 'your': 75,
 'also': 76,
 'chicago': 77,
 'back': 78,
 'got': 79,
 'by': 80,
 '

We can then use the tokenizer to convert all texts in the training-set to lists of these tokens.

In [0]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)

In [69]:

X_train[1]

'The only place inside the Loop that you can stay for $55/night. Also, the only place you can have a picnic dinner and get a little frisky on the 17th floor roof and then wake up in your room the next morning to an army of ants going in on your picnic leftovers.'

In [70]:
np.array(X_train_tokens[1])

array([   5,  773,   27, 1104, 1048,    7,   18,  408,    5,   27,   85,
       2071,    1, 3823,    9,    3,  301,    7,  385,    2,    5,    6,
        245,   33, 1348,   53,    5,  473,   27,    1, 2241,    4,   82,
         30,   80,    1,  869,    7,   54,   37,    7,  299,  337,    5,
         23,    4,  184,    5,    6,   95,  659,  203,  641,  326,    6,
        113,   16,   53,   24,  178,   12,   24,  146,    4,   43,    1,
         40,    5,  102,   98,  285,   30,    6,   64,  271,  296,   53,
         14,  337,    2,    5,  450,    2,  931,  165,  263,   19,  348,
         11,    6,   29, 1049,    4,   23,    4,  954,    8,    1,  507,
        162,    1,   22,    1,  182, 5096,   22,   81,    9,    3,  301,
          7,  201,    4,  153,   11,  590,  367,   73,   89,   11,  245,
         10,   46, 3371,    4,   31,    1, 1162,    6, 1112,    2,    1,
        251,   26,  145,  260,  346,   64,  446, 5320,    5, 1490,  149,
          3,  337,   30,  130,  178,    1,   40,   

We also need to convert the texts in the test-set to tokens.

In [0]:
X_test_tokens = tokenizer.texts_to_sequences(X_test)

Padding and Truncating Data¶
The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways of achieving this: (A) Either we ensure that all sequences in the entire data-set have the same length, or (B) we write a custom data-generator that ensures the sequences have the same length within each batch.

Solution (A) is simpler but if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.

So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.

In [0]:
num_tokens = [len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens = np.array(num_tokens)

The average number of tokens in a sequence is:

In [73]:
np.mean(num_tokens)

150.9283

The maximum number of tokens in a sequence is:

In [74]:
np.max(num_tokens)

949

The max number of tokens we will allow is set to the average plus 2 standard deviations.

In [75]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

406

In [76]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9527

When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.

So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network.

In [0]:
pad = 'pre'


In [0]:
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [0]:
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

We have now transformed the training-set into one big matrix of integers (tokens) with this shape:

In [81]:

X_train_pad.shape

(7000, 406)

The matrix for the test-set has the same shape:

In [83]:
X_test_pad.shape

(3000, 406)

Change of sequence of tokens into padded sequence.Note that when this is input to the Recurrent Neural Network, then it first inputs a lot of zeros. If we had padded 'post' then it would input the integer-tokens first and then a lot of zeros. This may confuse the Recurrent Neural Network.



In [84]:
np.array(X_train_tokens[1])
X_train_pad[1]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

**Tokenizer Inverse Map**

For some strange reason, the Keras implementation of a tokenizer does not seem to have the inverse mapping from integer-tokens back to words, which is needed to reconstruct text-strings from lists of tokens. So we make that mapping here.

In [0]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))


Helper-function for converting a list of tokens back to a string of words.

In [0]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [95]:
#For example, this is the original text from the data-set:

X_train[1]


'The only place inside the Loop that you can stay for $55/night. Also, the only place you can have a picnic dinner and get a little frisky on the 17th floor roof and then wake up in your room the next morning to an army of ants going in on your picnic leftovers.'

In [96]:
tokens_to_string(X_train_tokens[12])

"this is my review for the spa at dana i came here recently for a massage with i'm pretty sure that was her name and a facial with can't remember \xa0 the massage was amazing \xa0i had an 80 min swedish massage but she used all and really it towards my needs \xa0she was very friendly and talkative when i met her but knew when to stop talking when the massage began thank god nothing worse than a masseuse talking too much \xa0i have a really bad back and she really used deep pressure on those areas and wasnt afraid to get in there and work it lol the facial was good too i got a 50 min facial with it was my first time so the lady really educated me on everything she was talking a whole lot at first but i guess she kinda had to so i knew what was going on but then she let me relax so that was nice \xa0she gave a small massage on my neck and head it was really bad and pointless something she should work on no pressure at all the amenities sucked a tiny little steam room in the locker room s

**Create the Recurrent Neural Network**


In [0]:
model = Sequential()

In [0]:
embedding_size = 8


In [99]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Adding the gated GRU
model.add(GRU(units=16, return_sequences=True))

In [0]:
#Adding the second GRU with 8 output units
model.add(GRU(units=8, return_sequences=True))

In [0]:
# This code adds the third and final GRU with 4 output units. This will be followed by a dense-layer, so it should only give the final output of the GRU and not a whole sequence of outputs.

model.add(GRU(units=4))

dd a fully-connected / dense layer which computes a value between 0.0 and 1.0 that will be used as the classification output.

In [0]:
model.add(Dense(1#, activation='sigmoid'))


In [0]:
#Adam optimizer with the given learning-rate.
optimizer = Adam(lr=1e-3)

In [0]:
#Compile the Keras model so it is ready for training.
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [109]:

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 406, 8)            80000     
_________________________________________________________________
gru (GRU)                    (None, 406, 16)           1200      
_________________________________________________________________
gru_1 (GRU)                  (None, 406, 16)           1584      
_________________________________________________________________
gru_2 (GRU)                  (None, 406, 8)            600       
_________________________________________________________________
gru_3 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 83,545
Trainable params: 83,545
Non-trainable params: 0
_________________________________________________________________


**Train the Recurrent Neural Network**

We can now train the model. Note that we are using the data-set with the padded sequences. We use 5% of the training-set as a small validation-set, so we have a rough idea whether the model is generalizing well or if it is perhaps over-fitting to the training-set.

In [111]:
%%time
model.fit(X_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

Train on 6650 samples, validate on 350 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3


ValueError: ignored